## Implemetation of the heterologous pathway

As mentioned in the report, we need three reactions to allow for the production of 7-dehydrocholesterol in *S. cerevisiae*. In this file, we are adding the heterologous pathway to the model and saving that as a new file for subsequent use. 

In [1]:
#KERNEL 3.6!

# First, we can import some functions so we can use the model
from cobra.io import read_sbml_model, write_sbml_model
from cobra import Reaction, Metabolite
from cameo.strain_design import pathway_prediction

# Second, we can read the GEM and save it as ‘model’
model = read_sbml_model('data/yeastGEM.xml')

# Third, we can show general information about the loaded model
model

Name,M_yeastGEM_v8__46__4__46__2
Memory address,0x02253b225c70
Number of metabolites,2742
Number of reactions,4058
Number of groups,90
Objective expression,1.0*r_2111 - 1.0*r_2111_reverse_58b69
Compartments,"cell envelope, cytoplasm, extracellular, mitochondrion, nucleus, peroxisome, endoplasmic reticulum, Golgi, lipid particle, vacuole, endoplasmic reticulum membrane, vacuolar membrane, Golgi membrane, mitochondrial membrane"


First we can check if any pathways for lathosterol (a precursor) are available:

In [2]:
import cplex

ModuleNotFoundError: No module named 'cplex'

In [ ]:

import re
from cameo.strain_design import pathway_prediction
predictor = pathway_prediction.PathwayPredictor(model)
pathways = predictor.run(product="7-dehydrocholesterol", max_predictions=4)

Since there are none, we must add the reactions manually.

In [ ]:
# First reaction

new_reaction1 = Reaction('R07498')  # The first of the three reactions from above

# R07498 uses:
## Zymosterol + NADPH + H+ <=> 5alpha-Cholest-8-en-3beta-ol + NADP+
## We can find these as: (Note: All of these are in the cytoplasm)
### Zymosterol = s_1569[c]
### NADPH      = s_1212[c]
### H+         = s_0794[c]
### 
### 5alpha-Cholest-8-en-3beta-ol = Not yet defined
### NADP+                        = s_1207[c]

# Since 5alpha-Cholest-8-en-3beta-ol does not exist in the model, we will have to define it.
alphaCholest = Metabolite(id='5alpha-Cholest-8-en-3beta-ol_c', name='5alpha-Cholest-8-en-3beta-ol', compartment='c')


# We can now define the reaction stoichiometry
new_reaction1.add_metabolites({
                            model.metabolites.get_by_id("s_1569[c]"): -1,
                            model.metabolites.get_by_id("s_1212[c]"): -1,
                            model.metabolites.get_by_id("s_0794[c]"): -1,
                            alphaCholest: 1, # our newly created metabolites
                            model.metabolites.get_by_id("s_1207[c]"): 1
                             })

model.add_reactions([new_reaction1])

## Since the reaction can go both ways, we will set the bounds of the reaction to -1000 and 1000
#model.reactions.R07498.bounds = -1000, 1000

In [ ]:
# Here, we are testing if the intermediate compound can be produced
r_5a_exp = model.add_boundary(model.metabolites.get_by_id('5alpha-Cholest-8-en-3beta-ol_c'), type='demand')

with model:
    model.objective = r_5a_exp
    sol = model.optimize()
    print(sol.objective_value)

In [ ]:
# Second reaction

new_reaction2 = Reaction('R03353')  # The second of the three reactions from above

# R03353 uses:
## Lathosterol <=> 5alpha-Cholest-8-en-3beta-ol
## We can find these as: (Note: All of these are in the cytoplasm)
### 5alpha-Cholest-8-en-3beta-ol = alphaCholest
###
### Lathosterol                  = Not yet defined

# Since Lathosterol does not exist, we will have to define it.
Lathosterol = Metabolite(id='Lathosterol_c', name='Lathosterol', compartment='c')


# We can now define the reaction stoichiometry
new_reaction2.add_metabolites({
                            alphaCholest: -1, # The metabolite from above
                            Lathosterol: 1    # our newly created metabolites
                             })

model.add_reactions([new_reaction2])

## Since the reaction can go both ways, we will set the bounds of the reaction to -1000 and 1000
#model.reactions.R03353.bounds = -1000, 1000

In [ ]:
# Here, we are testing if the intermediate compound can be produced
r_la_exp = model.add_boundary(model.metabolites.get_by_id('Lathosterol_c'), type='demand')

with model:
    model.objective = r_la_exp
    sol = model.optimize()
    print(sol.objective_value)

In [ ]:
# Third reaction

new_reaction3 = Reaction('R07215')  # The third of the three reactions from above

## R07215 uses:
## Lathosterol + 2 Ferrocytochromeb5 + Oxygen + 2 H+ <=> 7-Dehydrocholesterol + 2 Ferricytochrome b5 + 2 H2O
## We can find these as: (Note: All of these are in the cytoplasm, except for the cytochromes, which are in the mitochondria)
### Lathosterol          = Lathosterol
### Ferrocytochrome b5   = s_0710[m]
### Oxygen               = s_1275[c]
### H+                   = s_0794[c]
###
### 7-Dehydrocholesterol = Not yet defined
### Ferricytochrome b5   = s_0709[m]
### H2O                  = s_0803[c]



# Since 7-Dehydrocholesterol does not exist, we will have to define it.
Dehydrocholesterol = Metabolite(id='7-Dehydrocholesterol_c', name='7-Dehydrocholesterol', compartment='c')


new_reaction3.add_metabolites({
                            Lathosterol: -1, # The metabolite we created above
                            model.metabolites.get_by_id("s_0710[m]"): -2,
                            model.metabolites.get_by_id("s_1275[c]"): -1,
                            model.metabolites.get_by_id("s_0794[c]"): -2,
                            Dehydrocholesterol: 1, # our newly created metabolites
                            model.metabolites.get_by_id("s_0709[m]"): 2,
                            model.metabolites.get_by_id("s_0803[c]"): 2
                             })

model.add_reactions([new_reaction3])

## Since the reaction can go both ways, we will set the bounds of the reaction to -1000 and 1000
#model.reactions.R07215.bounds = -1000, 1000

In [ ]:
r_7DH_export = model.add_boundary(model.metabolites.get_by_id('7-Dehydrocholesterol_c'), type='demand')

with model:
    model.objective = r_7DH_export
    sol = model.optimize()
    print(sol.objective_value)

From this, we can see that we are now able to produce 7-dehydrocholesterol in our cell factory!

The current state of the model is saved as "yeastGEM_het.xml" in the data folder.

In [ ]:
write_sbml_model(model, "data/yeastGEM_het.xml")

In [ ]:
model = read_sbml_model('data/yeastGEM_het.xml')

Let us try anaerobic production:

In [ ]:
with model:
    medium = model.medium
    medium['r_1992'] =0    
    model.medium = medium
    with model:
        model.objective = model.reactions.R07215
        DH_production = model.optimize().objective_value

print(DH_production)

That did not yield anything. Let's go back to aerobic production

In [ ]:
with model:
    model.objective = model.reactions.R07215
    DH_production = model.optimize().objective_value
    print("Max. 7-Dehydrocholesterol production [mmol gDW^-1 h^-1]:", DH_production)
    print("Theoretical max. yield [mmol-DH / mmol-glucose]:", DH_production / (-1*model.reactions.r_1714.flux))
    max_yield_mmol=DH_production / (-1*model.reactions.r_1714.flux)
    
